In [4]:
import collections.abc
import math
import os
from random import choice
from turtle import *
import numpy as np
import turtle
import random
import statistics
def nearest_pill_distance(tiles,nextmovestring,pacman):
    newpac = pacman.copy()
    for i in nextmovestring:
        if i=='U':
            newpac[1]+=1
        if i=='D':
            newpac[1]-=1
        if i=='R':
            newpac[0]+=1
        if i=='L':
            newpac[0]-=1
    d0=10000
    pi = newpac[0]
    pj = newpac[1]
    for i in range(0,20):
        for j in range(0,20):
            if tiles[i,j]==2:
                d1 = math.sqrt((i-pi)**2+(j-pj)**2)
                if d0>d1:
                    d0=d1
    return int(round(d0))
    
def pills_in_way(tiles,nextmovestring,pacman):
    point=0
    newpac = pacman.copy()
    for i in nextmovestring:
        if i=='U':
            newpac[1]+=1
        if i=='D':
            newpac[1]-=1
        if i=='R':
            newpac[0]+=1
        if i=='L':
            newpac[0]-=1
        pi = newpac[0]
        pj = newpac[1]
        
        if -1<pi<19 and -1<pj<19 :
            if tiles[pi,pj]==2:
                point+=100
                
    return int(point)
def nearest_ghost_distance(tiles,nextmovestring,pacman,ghosts):
    newpac = pacman.copy()
    for i in nextmovestring:
        if i=='U':
            newpac[1]+=1
        if i=='D':
            newpac[1]-=1
        if i=='R':
            newpac[0]+=1
        if i=='L':
            newpac[0]-=1
    d0=10000
    pi = newpac[0]
    pj = newpac[1]
    for p,c in ghosts:
        d1 = math.sqrt((pi-p[0])**2+(pj-p[1])**2)
        if d0>d1:
            d0=d1
    return int(round(d0))
def walls_in_way(tiles,nextmovestring,pacman):
    point=0
    newpac = pacman.copy()
    for i in nextmovestring:
        if i=='U':
            newpac[1]+=1
        if i=='D':
            newpac[1]-=1
        if i=='R':
            newpac[0]+=1
        if i=='L':
            newpac[0]-=1
        pi = newpac[0]
        pj = newpac[1]
        if -1<pi<19 and -1<pj<19:
            if tiles[pi,pj]==0 :
                point=-1000000
                return int(point)

        else:
            p=-1000000
            return int(p)
    return 0
def ghosts_in_way(tiles,nextmovestring,ghosts,pacman):
    point=0
    newpac = pacman.copy()
    for i in nextmovestring:
        if i=='U':
            newpac[1]+=1
        if i=='D':
            newpac[1]-=1
        if i=='R':
            newpac[0]+=1
        if i=='L':
            newpac[0]-=1
        pi = newpac[0]
        pj = newpac[1]
        if -1<pi<19 and -1<pj<19:
            k=0
            for p,c in ghosts:
                k+=1
                if p[0] == pi and p[1] == pj:
                    if k<3:
                        point=-1000000
                        return int(point)
                    else:
                        point=-10
                    

        else:
            p=-1000000
            return int(p)
    return int(point)
#fit_eval function to evaluate the fitness of answers

def Gp_fit_eval(Population,N,tiles,pacman):
    fit = []
    for i in range (0,N):    
        c = 0
        X = Population[i]
        f1 = 1000 - 10 * int(nearest_pill_distance(tiles,X,pacman))
        f2 = int(pills_in_way(tiles,X,pacman))
        f3 = int(nearest_ghost_distance(tiles,X,pacman,ghosts)) -10
        f4 = int(walls_in_way(tiles,X,pacman))
        f5 = int(ghosts_in_way(tiles,nextmovestring,ghosts,pacman))
        f = f1+f2+f3+f4+f5
        fit.append(f)
    
    avg_fit = statistics.mean(fit)
    max_loc = np.argmax(fit)
    max_fit = fit[max_loc]
    opt_sol = Population[max_loc]
    if len(opt_sol)==0:
        return fit,avg_fit,fit[1],Population[1]
    return fit,avg_fit,max_fit,opt_sol

def Gp_roulette_wheel(population,N,fit,opt_sol):
    mating_pool = population.copy()
    for i in range(4,N):
        if fit[i]<0:            
            mating_pool[i]=opt_sol
            break
    return mating_pool
def Gp_crossover(mating_pool,Pc,N):
    off1=[]
    new_pop = mating_pool.copy()
    parent_num = np.random.permutation(N)
    for j in range (0,N-1,2):
        pointer1 = parent_num[j]
        pointer2 = parent_num[j+1]
        p1 = mating_pool[pointer1]
        p2 = mating_pool[pointer2]        
        lenp1 = len(p1)
        lenp2 = len(p2)
        cut_point11 = random.randint(1,lenp1-1)    
        cut_point12 = random.randint(cut_point11,lenp1-1)
        cut_point21 = random.randint(0,lenp2-2)    
        cut_point22 = random.randint(cut_point21+1,cut_point21+3)
        if random.random()<Pc:
            
            off1.append(p1[0:cut_point11])
            off1.append(p2[cut_point21+1:cut_point22])
            off1.append(p1[cut_point12+1:lenp1])
        npop=[]
        for i in off1:
            for h in i:
                npop.append(h)

        if len(npop) < 2 :
            npop = p1.copy()
        
        if len(npop) > 8 :
            t=[]
            t = npop[0:5]
            npop = t.copy()
        new_pop[j] = npop
        new_pop[j+1] = p2
        
        new_pop[0]=['U','U']
        new_pop[1]=['D','D']
        new_pop[2]=['R','R']
        new_pop[3]=['L','L']
                    
                    

    

            
    return new_pop
def pacman_controller(tiles,pacman,ghosts,nextmovestring,movenum):
    lastchoice = nextmovestring[movenum:len(nextmovestring)]
    newchoice = gp_(tiles,pacman,ghosts)
    Population = np.array([lastchoice,newchoice],dtype=object)
    a,b,c,d = Gp_fit_eval(Population,2,tiles,pacman)
    nextmovestring = d
    movenum = 1
    if nextmovestring[0]=='U':
        aim = np.array([0,1])
    if nextmovestring[0]=='D':
        aim = np.array([0,-1])
    if nextmovestring[0]=='R':
        aim = np.array([1,0])
    if nextmovestring[0]=='L':
        aim = np.array([-1,0])
     
    return nextmovestring , movenum , aim
def gp_(tiles,pacman,ghosts):
    
    Population = np.array([['U','U'],
                           ['D','D'],
                           ['R','R'],
                           ['L','L'],
                           ['U','R','R'],
                           ['D','L','D'],
                           ['R','R','U'],
                           ['L','D','U'],
                           ['U','L','U'],
                           ['D','R','U'],
                           ['R','U','L'],
                           ['D','D','U'],
                           ['R','R','U'],
                           ['L','L','U'],
                           ['U','R','R'],
                           ['D','L','D'],
                           ['R','R','U'],
                           ['L','D','U'],
                           ['U','L','U'],
                           ['D','R','U'],
                           ['R','U','L'],
                           ['U','L','U'],
                           ['D','R','U'],
                           ['R','U','L'],
                           ['D','D','U'],
                           ['R','R','U'],
                           ['L','L','U'],
                           ['U','R','R'],
                           ['D','L','D'],
                           ['R','R','U'],
                           ['L','D','U'],
                           ['U','L','U'],
                           ['D','R','U'],
                           ['D','R','U'],
                           ['R','U','L'],
                           ['D','D','U'],
                           ['R','R','U'],
                           ['L','L','U'],
                           ['U','R','R'],
                           ['D','L','D'],
                           ['R','R','U'],
                           ['L','D','U'],
                           ['U','L','U'],
                           ['D','R','U'],
                           ['D','R','U'],
                           ['R','U','L'],
                           ['D','D','U'],
                           ['R','R','U'],
                           ['L','L','U'],
                           ['U','R','R'],
                           ['D','L','D'],
                           ['R','R','U'],
                           ['L','D','U'],
                           ['U','L','U'],
                           ['D','R','U'],
                           ['D','R','U'],
                           ['R','U','L'],
                           ['L','U','L']], dtype=object)
    popback= Population.copy()
    N=55
    Pc=0.6
    Pm=0.25
    Rep_Num=20
    #     ___________________________________________________________

    Best_So_Far=[]
    Average_fitness=[]
    for R in range(0,Rep_Num):
    
        fit,avg_fit,max_fit,opt_sol = Gp_fit_eval(Population,N,tiles,pacman)
        if R==0:
            Best_So_Far.append(max_fit)
            final_sol = opt_sol
        elif max_fit > Best_So_Far[R-1]:
            Best_So_Far.append(max_fit)
            final_sol = opt_sol
        else:
            Best_So_Far.append(Best_So_Far[R-1])
        
        Average_fitness.append(avg_fit)
        mating_pool = Gp_roulette_wheel(Population,N,fit,opt_sol)
        new_pop = Gp_crossover(mating_pool,Pc,N)
        Population = new_pop.copy()
    global nextmovestring
    nextmovestring = final_sol 
    return nextmovestring

In [10]:
#main program starts here:

state = {'score': 0 , 'time':200}
map_x = 400
map_y = 400
dw=0.15
aim = np.array([1,0])
pacman = np.array([4, 4])
nextmovestring = ['L','U','U','L']
movenum=0
ghosts = np.array([
    [[2, 17], [1, 0]],
    [[9,14], [0, -1]],
    [[17,17], [-1, 0]]
])

tiles =np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
])

for i in range(2,18):
    for j in range(2,18):
        if random.random()<dw:
            tiles[i,j]=0
        else: tiles[i,j]=1

def world():
    "Draw world"
    bgcolor('black')
    color('Cyan')
    
    for x in range(int(map_x/20)):        
        for y in range(int(map_y/20)):               
            if tiles[x,y] == 0:
                pu()
                goto(x*20+10,y*20+10)
                dot(20, 'Cyan') #wall

            if tiles[x,y] == 1 and random.random()<0.5:
                pu()
                goto(x*20 + 10, y*20 + 10)
                dot(5, 'deep pink') #pills
                tiles[x,y] = 2
                
                
def move():
    "Move pacman and ghosts"
    global pacman
    global aim
    global ghosts
    global nextmovestring
    global tiles
    global movenum
    p0 = pacman.copy()
    ghosts0 = ghosts.copy()
    A,B,C = pacman_controller(tiles,pacman,ghosts0,nextmovestring,movenum)
    aim = C.copy()
    movenum = B
    nextmovestring = A.copy()
    p1 = pacman + aim

    if p1[0]<19 and p1[1]<19:
        if tiles[p1[0] , p1[1]] > 0:
            pacman = pacman + aim
           
    i = pacman[0]
    j = pacman[1]
    
    if tiles[i,j] == 2:
        tiles[i,j] = 1
        state['score'] += 1
        pu()
        goto(-15, 360)
        dot(30, 'black')
        pu()
        goto(-10, 350)
        color('deep pink')
        write(f"{state['score']}", move=True, align="right", font=("Arial", 14, "normal"))
    
    pu()
    goto(p0[0]*20 + 10, p0[1]*20 + 10)
    dot(20, 'black')
    pu()
    goto(i*20 + 10, j*20 + 10)
    dot(20, 'yellow')

    
    gh=-1

    for point, course in ghosts:
        up()
        goto(point[0]*20 + 10, point[1]*20 + 10)
        dot(20, 'black')
        if tiles[point[0] , point[1]] ==2:
            up()
            goto(point[0]*20 + 10, point[1]*20 + 10)
            dot(5, 'deep pink') 
        gh+=1
        p = point+ course
        if tiles[p[0] , p[1]] > 0:
            point = point + course
            
        else:
            options = [
                (1, 0),
                (-1, 0),
                (0, 1),
                (0, -1),
            ]
            plan = choice(options)
            course[0] = plan[0]
            course[1] = plan[1]
            p = point+ course
            if tiles[p[0] , p[1]] >0:
                point = point + course
        ghosts [gh,0]=point


        up()
        goto(point[0]*20 + 10, point[1]*20 + 10)
        dot(20, 'red')
   
    for pgh,cgh in ghosts:
        pa = pacman*20
        gho = pgh*20
        A = math.sqrt(((pa[0]-gho[0])**2)+((pa[1]-gho[1])**2))
        if  A  < 25:
            pu()
            goto(-200, 300)
            color('deep pink')
            write("you lost", move=True, align="right", font=("Arial", 14, "normal"))
            return
    print(nextmovestring)
    state['time'] -= 1
    pu()
    goto(-20, 210)
    dot(40, 'black')
    pu()
    goto(-10, 210)
    color('deep pink')
    write(f"{state['time']}", move=True, align="right", font=("Arial", 14, "normal"))
    
    ontimer(move, 10)

            

tracer(False)

pu()
goto( -30, 350)
color('deep pink')
write("score:", move=True, align="right", font=("Arial", 14, "normal"))
pu()
goto(-10, 350)
color('deep pink')
write(f"{state['score']}", move=True, align="right", font=("Arial", 14, "normal"))
pu()
goto( -50, 210)
color('deep pink')
write("time:", move=True, align="right", font=("Arial", 14, "normal"))
pu()
world()
move()
done()

['R', 'U', 'R', 'R', 'D', 'U', 'U', 'L']
['R', 'U', 'U', 'R', 'D', 'U', 'D', 'D']
['D', 'D', 'U', 'L', 'D']
['D', 'U', 'L', 'D']
['L', 'U', 'L', 'L', 'D']
['U', 'L', 'L', 'D']
['R', 'U', 'R', 'U', 'D', 'R', 'R', 'U']
['U' 'R' 'U' 'D' 'R' 'R' 'U']
['R' 'U' 'D' 'R' 'R' 'U']
['R', 'U', 'L', 'U', 'R', 'U', 'U', 'U']
['U' 'L' 'U' 'R' 'U' 'U' 'U']
['U', 'L', 'R', 'L', 'U', 'D', 'R', 'U']
['U', 'L', 'U', 'D', 'U', 'D', 'R', 'U']
['L', 'U', 'D', 'U', 'D', 'R', 'U']
['U', 'D', 'U', 'D', 'R', 'U']
['R', 'D', 'D', 'D', 'L', 'D', 'R', 'D']
['L', 'U', 'U', 'L', 'L', 'U', 'U', 'D']
